# Deploying an Air Travel Agent on AIPC

## Introduction

This notebook showcases on deploying local LLM agents using the Langchain tools on Intel® Core™ Ultra Processors. The aim is to deploy an Air Travel Agent on the iGPU (integrated GPU) of the AIPC. For this, Llamacpp GPU backend is setup and the agent created using the local LLM model. The agent makes use of langchain toolkits and tools for travel related queries. 


### Table of Contents
1. Creating the Air Travel Agent
2. Deploying with Gradio

## Creating the Air Travel Agent

In [1]:
import os 
from langchain.agents import tool, AgentExecutor, create_react_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description_and_args

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
#Selecting the local LLM model
import ipywidgets as widgets

selected_model= widgets.Dropdown(
    options=['llama-2-7b.Q4_K_S.gguf',
            'Meta-Llama-3.1-8B-Instruct-Q4_K_S.gguf'],
    value='llama-2-7b.Q4_K_S.gguf',
    description='Model:',
    disabled=False
)

selected_model

Dropdown(description='Model:', index=1, options=('llama-2-7b.Q4_K_S.gguf', 'Meta-Llama-3.1-8B-Instruct-Q4_K_S.…

In [4]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


llm = LlamaCpp(
    model_path="models/"+ selected_model.value,
    n_gpu_layers=-1, # Uncomment to use GPU acceleration
    seed=512, # Uncomment to set a specific seed
    n_ctx=4096, # Uncomment to increase the context window
    f16_kv=True, # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False, 
    temperature=0,
 
)
llm.client.verbose = False #for silencing llm.invoke()

In [5]:
from langchain.agents import load_tools

tools= load_tools(["serpapi"])

In [6]:
# Prompt template for the agent
from langchain_core.prompts import PromptTemplate

#Answer the following questions as best you can.

template = '''
1. You are an AI assistant which is tasked to give expert advices to questions on travel and tourism. 
2. The context is that user is expecting you to answer questions on travel, flights and tourism recommendations.
3. You will always rely on your knowledge base in the first place
4. You can also take the help of following tools
{tools} 
5. For Flight search queries use the search tool only
6. If the above tools does not help, try to use your general knowledge for one more time
7. If you can't find relevant information, answer that you don't know and    


Use the following format 

Question: the input question you must answer
Thought: you should always think about what to do
Action: [{tool_names}] are available. 
Action Input: the input to the action
Observation: the result of the action
IMPORTANT: ... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer. Stop Thinking.
Final Answer: the final answer to the original input question in bullet points

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [7]:
from langchain.agents import create_react_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
agent = create_react_agent(
    llm,
    tools,
    prompt,
    tools_renderer=render_text_description_and_args,
    output_parser=ReActJsonSingleInputOutputParser(),
    
)

In [8]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
    max_execution_time=120,
    #max_iterations=3,
)

In [ ]:
agent_executor.invoke(
    {
        "input": "What are the top 3 places to visit in Paris"
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "What is the distance to eiffel tower from the airport"
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "when is the best time of the year to visit Paris"
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "What is the name of the airport in paris and what is the airport code of it"
    }
)


In [ ]:
agent_executor.invoke(
    {
        "input": "What are the flights available from Paris, France to New York, USA in the first week of October 2024."
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "What is the cheapest flight available from Paris, France to New York, USA in the first week of October 2024. Please give the flight details"
    }
)

## Deploying with Gradio

In [2]:
import warnings
warnings.filterwarnings('ignore')
import gradio as gr

In [ ]:
def handle_prompt(user_prompt, history):
    final_answer = agent_executor.invoke( {"input": user_prompt})
    return final_answer['output']


examples = [
    "What are the top attractions in Dubai",
    "When is the best time of the year to visit Dubai",
    "What are the available flights from New york to Dubai in the first week of December 2024",
    "How tall is Burj Khalifa",    
]

demo = gr.ChatInterface(fn=handle_prompt, examples=examples, title="Hi, I'm an Air Travel Agent")
demo.launch()